In [1]:
# !pip install ./sacremoses/sacremoses-master/
# !pip install ./transformers/transformers-master/

STRIDE = 1
def is_jupyter():
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return True
        
    except:
        return False

### Required Imports

I've added imports that will be used in training too

In [2]:
from sklearn.model_selection import train_test_split
import re
from random import shuffle as shfl
import random

from auc import MyAUCCallback
from sklearn.utils import shuffle
from datetime import datetime
import pandas as pd
pd.set_option('display.max.columns', 500)
import numpy as np
import os
# os.environ['CUDA_VISIBLE_DEVICES'] ='3'
# os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-10.1/lib64'
import matplotlib.pyplot as plt
import gc
from shutil import copyfile
from catalyst.dl import SupervisedRunner, AlchemyLogger, CriterionCallback
from catalyst.dl.callbacks.metrics import AUCCallback
from torch.utils.data import DataLoader, SubsetRandomSampler,Dataset
batch_size =32
token = "d1dd16f08d518293bcbeddd313b49aa4"
DATA_DIR = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'

/home/kb/jig_env/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/kb/jig_env/lib/python3.7/site-packages/graphql/type/directives.py:55: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Py

In [3]:
if os.uname()[1] == 'kb-Z370P-D3':
    # desktop
    LOG_PATH = '/media/ssd/logs/jigsaw'
    SERVER = False
    print('Working on desktop')
elif os.uname()[1] == 'kb-server':
    # server
    LOG_PATH = '/home/kb/logs/jigsaw'
    SERVER = True
    print('Working on server')
else:
    raise Exception('which hostname???')
    

Working on desktop


In [4]:
df_test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')\
    .rename(columns={'content':'comment_text'})

In [5]:
df_pseudo = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/pseudo_labelling_09393.csv')
df_pseudo2 = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/pseudo_labelling_09459.csv')
df_pseudo = df_pseudo.merge(df_pseudo2, on='id')
df_pseudo = df_pseudo[((df_pseudo['toxic_x']>0.9) & (df_pseudo['toxic_y']>0.9)) | ((df_pseudo['toxic_x']<0.1) & (df_pseudo['toxic_y']<0.1))]
df_pseudo['toxic'] = (df_pseudo['toxic_y'] + df_pseudo['toxic_x'])/2
df_pseudo = df_pseudo[['id', 'toxic']]
df_pseudo.loc[df_pseudo['toxic']>0.5, 'toxic'] = 1.0
df_pseudo.loc[df_pseudo['toxic']<=0.5, 'toxic'] = 0.0
df_pseudo = df_pseudo.merge(df_test, on='id').rename(columns={'content':'comment_text'})

In [6]:
df_pseudo.head()

,id,toxic,comment_text,lang
0,0,0.0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,0.0,"Вполне возможно, но я пока не вижу необходимо...",ru
2,3,0.0,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
3,4,0.0,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr
4,5,0.0,Le truc le plus important dans ta tirade c est...,fr


In [7]:
len(df_pseudo)

42421

In [8]:
df_pseudo.groupby('toxic', as_index=False)['id'].count()

,toxic,id
0,0.0,39129
1,1.0,3292


In [9]:
df_valid_3lng = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [10]:
df_valid_3lng.groupby('toxic', as_index=False)['id'].count()

,toxic,id
0,0,6770
1,1,1230


In [11]:
df_valid_3lng.head()

,id,comment_text,lang,toxic
0,0,Este usuario ni siquiera llega al rango de ...,es,0
1,1,Il testo di questa voce pare esser scopiazzato...,it,0
2,2,Vale. Sólo expongo mi pasado. Todo tiempo pasa...,es,1
3,3,Bu maddenin alt başlığı olarak uluslararası i...,tr,0
4,4,Belçika nın şehirlerinin yanında ilçe ve belde...,tr,0


In [12]:
len(df_valid_3lng)

8000

In [13]:
df_pseudo = df_pseudo.append(df_valid_3lng)

In [14]:
target_column = 'toxic'

### Define dataset

In [15]:
# 7from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import time
from torch.optim import lr_scheduler

import torch
from tqdm import tqdm
#import torch.utils.data as data
from torchvision import datasets, models, transforms
from transformers import *
import random
from math import floor, ceil
from sklearn.model_selection import GroupKFold

MAX_LEN = 96#192#192#512
SEP_TOKEN_ID = 102

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True, train_transforms=None):
        
        self.train_transforms = train_transforms
        self.df = df
        if train_mode:
            self.labels = df.toxic.values
            self.toxic_inds = np.where(self.labels==1)[0]
            self.normal_inds = np.where(self.labels==0)[0]            
            
            
            print(f'Here is {len(self.labels)} samples, {len(self.toxic_inds)} samples and {len(self.normal_inds)} samples')
            print(f'Class balance is {len(self.toxic_inds)/len(self.labels):.2f}')
            
        self.train_mode = train_mode
        self.labeled = labeled

        self.tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')#, 


    def __getitem__(self, index):
        row = self.df.iloc[index]
        token_ids = self.get_token_ids(row)
        
        if self.labeled:
            labels = self.get_label(row)
            return {'features': token_ids, 'targets': labels}

        else:
            return {'features': token_ids}

    def __len__(self):
        return len(self.df)


        
    def get_token_ids(self, row):
        
        text = row.comment_text
        if self.train_transforms:
            lang = 'en' if 'lang' not in list(row.keys().values) else row.lang
            text, _ = self.train_transforms(data=(row.comment_text, lang))['data']
        
        if self.train_mode:
            token_ids = self.tokenizer.encode(text, max_length=1024)
            if len(token_ids) < MAX_LEN:
                ids = torch.tensor(token_ids + [0] * (MAX_LEN - len(token_ids)))
            else:
                ind_beg = random.randint(0, len(token_ids)-MAX_LEN)
                ids = torch.tensor(token_ids[ind_beg:ind_beg+MAX_LEN])
        else:
            token_ids = self.tokenizer.encode(text, max_length=MAX_LEN)
            if len(token_ids) < MAX_LEN:
                ids = torch.tensor(token_ids + [0] * (MAX_LEN - len(token_ids)))
            else:
                ids = torch.tensor(token_ids[:MAX_LEN])

        return ids

    def get_label(self, row):
        label = np.round(row[target_column])
        return torch.tensor([1-label, label]).float()
    
    def collate_fn(self, batch):
        token_ids = torch.stack([x[0] for x in batch])

        if self.labeled:
            labels = torch.stack([x[1] for x in batch])
            return {'features': token_ids, 'targets': labels}
        else:
            return {'features': token_ids}
        
 

## Build Model

In [16]:
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F


class QuestModel(nn.Module):
    def __init__(self, n_classes=2):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
    
        self.bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-large')

        self.fc = nn.Linear(1024, n_classes)

    def forward(self, ids):

        layers = self.bert_model(input_ids=ids)
        out = F.dropout(layers[0][:, 0, :], p=0.2, training=self.training)
        logit = self.fc(out)
        return logit
    


In [17]:
# bert_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

In [18]:
class BalancedSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset):
       
        self.toxic_inds = dataset.toxic_inds.copy()
        self.normal_inds = dataset.normal_inds.copy()
        
        self.num_samples = 2*max(len(self.toxic_inds), len(self.normal_inds))
        
        random.Random(12345).shuffle(self.toxic_inds)
        random.Random(12345).shuffle(self.normal_inds)

        
        self.inds = []
        for i in range(max(len(self.toxic_inds), len(self.normal_inds))):
            self.inds.append(self.normal_inds[i%len(self.normal_inds)])
            self.inds.append(self.toxic_inds[i%len(self.toxic_inds)])

    def __iter__(self):
        #print ('\tcalling Sampler:__iter__')
        return iter(self.inds)

    def __len__(self):
        #print ('\tcalling Sampler:__len__')
        return self.num_samples

class BalancedSamplerRnd(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset):
       
        self.toxic_inds = dataset.toxic_inds.copy()
        self.normal_inds = dataset.normal_inds.copy()
        
        self.num_samples = 2* len(self.normal_inds)
        
        random.Random(12345).shuffle(self.toxic_inds)
        random.Random(12345).shuffle(self.normal_inds)
        
        N = int(len(self.normal_inds) / len(self.toxic_inds))
        generated_seq = []
        for k in range(N):
            tmp = list(range(len(self.toxic_inds)))
            random.Random(12345).shuffle(tmp)
            generated_seq += tmp 
            
        generated_seq += tmp[:len(self.normal_inds)-len(generated_seq)]
        
        self.inds = []
        for i in range(len(self.normal_inds)):
            self.inds.append(self.normal_inds[i])
            self.inds.append(self.toxic_inds[generated_seq[i]])

    def __iter__(self):
        #print ('\tcalling Sampler:__iter__')
        return iter(self.inds)

    def __len__(self):
        #print ('\tcalling Sampler:__len__')
        return self.num_samples


In [19]:
def get_loaders(to_balance=True, shuffle_before=True):
    if SERVER:
        workers=1
    else:
        workers = 8    
    
    

    train_dataset = QuestDataset(df_pseudo, train_mode=True)#, train_transforms=get_train_transforms())
    valid_dataset = QuestDataset(df_valid_3lng, train_mode=False)
    
       
    train_loader = DataLoader(
        train_dataset,
        num_workers=workers,
        sampler=BalancedSamplerRnd(train_dataset) if to_balance else None,
        batch_size=batch_size,
    )
    valid_loader = DataLoader(
        valid_dataset,
        num_workers=workers,
        batch_size=batch_size,
    )    
    
       
    loaders = {}
    loaders['train'] = train_loader
    loaders['valid'] = valid_loader
    
    
    for i in ['es', 'it', 'tr']:
        df = df_valid_3lng
        df = df[df['lang']==i]

        loaders['valid_'+ i] = DataLoader(
            QuestDataset(df, train_mode=False),
            num_workers=workers,
            batch_size=batch_size,
        )
    
    
    return loaders


In [20]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [22]:
def freeze_until(net, param_name):
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name

def train_from_to(net, param_name_from, param_name_to):
    
    found_1_name = False
    found_2_name = False
    train_params = False
    
    for name, params in net.named_parameters():
        
        if not found_1_name:
            if name == param_name_from:
                train_params = True
                found_1_name = True
            
        elif not found_2_name:
            if name == param_name_to:
                train_params = False
                found_2_name = True
#         print(f'layer {name} will train? - {train_params}')
        params.requires_grad = train_params
        
# param_name = 'bert_model.encoder.layer.10.attention.self.query.weight'
param_name_from = 'bert_model.encoder.layer.8.attention.self.query.weight'
param_name_to = 'bert_model.encoder.layer.16.attention.self.query.weight'

CHECKPOINT = '/media/ssd/logs/jigsaw/js_v14_engbias/05082020_084031_lr1e-06_2stage_16lr/simple/checkpoints/best.pth'

project = "js_v14_engbias"
num_epochs = 2

group = datetime.now().strftime("%m%d%Y_%H%M%S")

    
if SERVER:
    group = f'{group}_srv'
    
if STRIDE > 1:
    group = f'{group}_str{STRIDE}'


gradient_accumulation_steps = 1

lr = 1e-6#1e-5#0.0001
group += f'_lr{lr}_3stage_16lr'

group = group.replace('.', '')

runner = SupervisedRunner(input_key=('features'), input_target_key=('targets'), output_key=('logits'))

experiment = 'simple'

torch.cuda.empty_cache()

logdir = f"{LOG_PATH}/{project}/{group}/{experiment}"

model = QuestModel(2)

checkpoint = torch.load(CHECKPOINT)#, map_location=device)   
model.load_state_dict(checkpoint['model_state_dict'])
del checkpoint

freeze_until(model, param_name_from)
# train_from_to(model, param_name_from, param_name_to)
# model = model.to(device)


loaders = get_loaders(to_balance=True)


t_total = len(loaders['train'])//gradient_accumulation_steps*num_epochs
warmup_proportion = 0.01
num_warmup_steps = t_total * warmup_proportion

#     criterion = torch.nn.BCEWithLogitsLoss()
criterion = FocalLoss(alpha=0.2, gamma=1.5, logits=True, reduce=True)
#     criterion = torch.nn.CrossEntropyLoss()
#     criterion = torch.nn.BCELoss()
optimizer = AdamW(model.parameters(), lr = lr)
#    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=t_total) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.25)
print(f'----------------Experiment: {experiment}')

runner.train(
    fp16=dict(opt_level="O1") ,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
    verbose=True,
    distributed=False if is_jupyter() else True,
    callbacks=[
        AlchemyLogger(
                token=token, # your Alchemy token
                project=project,
                experiment=experiment,
                group=group,
            ),
        MyAUCCallback()

    ],
    main_metric='auc/_mean',
    minimize_metric=False,
    
    #fp16={"opt_level": "O1"}

)

Here is 50421 samples, 4522 samples and 45899 samples
Class balance is 0.09
----------------Experiment: simple
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
1/2 * Epoch (train):   0% 1/2869 [00:00<26:42,  1.79it/s, loss=0.009]

/home/kb/jig_env/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning:

size_average and reduce args will be deprecated, please use reduction='none' instead.

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning:

This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)



1/2 * Epoch (train):  29% 842/2869 [03:10<07:34,  4.46it/s, loss=0.006]    Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
1/2 * Epoch (train): 100% 2869/2869 [10:55<00:00,  4.38it/s, loss=0.002]    
1/2 * Epoch (valid): 100% 250/250 [00:18<00:00, 13.73it/s, loss=0.018]
1/2 * Epoch (valid_es): 100% 79/79 [00:05<00:00, 13.25it/s, loss=0.055]
1/2 * Epoch (valid_it): 100% 79/79 [00:05<00:00, 13.36it/s, loss=8.430e-05]
1/2 * Epoch (valid_tr): 100% 94/94 [00:07<00:00, 13.28it/s, loss=0.003]    
[2020-05-28 22:51:15,955] 
1/2 * Epoch 1 (_base): lr=1.000e-06 | momentum=0.9000
1/2 * Epoch 1 (train): auc/_mean=0.9941 | auc/class_0=0.9941 | loss=0.0073
1/2 * Epoch 1 (valid): auc/_mean=0.9706 | auc/class_0=0.9706 | es_auc/_mean=0.9606 | es_auc/class_0=0.9606 | es_loss=0.0364 | it_auc/_mean=0.9521 | it_auc/class_0=0.9521 | it_loss=0.0387 | loss=0.0258 | tr_auc/_mean=0.9939 | tr_auc/class_0=0.9939 | tr_loss=0.0059
1/2 * Epoch 1 (valid_es): auc/_mean=0.9606 | auc/clas

In [23]:
# for i, row in df_train.iterrows():
#     print(list(row.keys().values))
#     break

In [24]:
# --

In [25]:
# CHECKPOINT = '/media/ssd/logs/jigsaw/js_v14_engbias/05232020_131312_lr1e-07_3stage_16lr/simple/checkpoints/train.1.pth'

In [26]:
# model = QuestModel(2)

# checkpoint = torch.load(CHECKPOINT)#, map_location=device)   

# model.load_state_dict(checkpoint['model_state_dict'])
# del checkpoint

In [27]:
# model = model.to(device)
_ = model.eval()

In [28]:
df_test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')\
    .rename(columns={'content':'comment_text'})
ds = QuestDataset(df_test, train_mode=False, labeled=False)
loader = DataLoader(
        ds,
        num_workers=8,
        batch_size=batch_size,
    )    

In [29]:
df_test.head()

,id,comment_text,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr


In [30]:
y_pred = torch.zeros( len(ds), 2).to(device)

j = 0
with torch.no_grad():
    for data in tqdm(loader):
        x = data['features'].to(device)
        thish_batch_size = len(x)            
        
        
        y_pred[j:j+thish_batch_size] = model(x)
            
        j += thish_batch_size

100%|██████████| 1995/1995 [02:18<00:00, 14.45it/s]


In [31]:
from torch import sigmoid
y_pred_sigmoid = sigmoid(y_pred)

In [32]:
y_pred_sigmoid_one_np = y_pred_sigmoid.cpu().numpy()

In [33]:
df_sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [34]:
df_sub['toxic'] = y_pred_sigmoid_one_np[:,1]

In [35]:
df_sub.to_csv(f'submission_v1421_NEWBRANCH_valpsd_train_{num_epochs}ep_lr{lr}.csv', index=False)

In [36]:
df_sub

,id,toxic
0,0,0.014785
1,1,0.066206
2,2,0.562777
3,3,0.013689
4,4,0.018725
...,...,...
63807,63807,0.661486
63808,63808,0.010368
63809,63809,0.099295
63810,63810,0.008985
